<a href="https://colab.research.google.com/github/Ihoold/pixelgen/blob/master/funit_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from unittest import mock
import os
import numpy as np
import tensorflow as tf
import json
import matplotlib.pyplot as plt
from tensorflow.python.estimator import estimator
from google.colab import drive, auth

In [0]:
R, C = 4, 3
EXAMPLES = R * C
CHANNELS = 4

ALPHA = 0.2
BATCH_SIZE = 1024
EPOCHS = 20000
EVAL_EPOCHS = 1000
G_LR = 0.0002
D_LR = 0.0001
LR_REC = 0.1
LR_FEAT = 1
KERNEL_SIZE = 4
NUM_CLASSES = 21

MODEL_NAME = 'FUNIT'
RUN_NAME = 'FUNIT_only_disc_1'

data_train_file = 'gs://tputestingmnist/datasets/dataset_train_funit.tfrecords'
data_test_file = 'gs://tputestingmnist/datasets/dataset_test_funit.tfrecords'
MODEL_DIR = 'gs://tputestingmnist/{}/{}/'.format(MODEL_NAME, RUN_NAME)
GOOGLE_DRIVE_DIR = '/content/gdrive/My Drive/Programowanie/PixelGen/{}'.format(RUN_NAME)
TF_MASTER = 'grpc://{}'.format(os.environ['COLAB_TPU_ADDR'])

In [0]:
#################################### SETUP #####################################

def setup():
    drive.mount('/content/gdrive')
    auth.authenticate_user()


def upload_credentials():
    # Upload credentials to TPU.
    with tf.Session(TF_MASTER) as sess:    
        with open('/content/adc.json', 'r') as f:
            auth_info = json.load(f)
        tf.contrib.cloud.configure_gcs(sess, credentials=auth_info)

In [0]:
################################# DATA INPUT ###################################

def parser(serialized_example):
        """Parses a single example into image and label tensors."""
        features = tf.parse_single_example(
            serialized_example,
            features={
                'image_content': tf.FixedLenFeature([], tf.string),
                'image_style': tf.FixedLenFeature([], tf.string),
                'label': tf.FixedLenFeature([], tf.int64) 
            })

        content_image = tf.decode_raw(features['image_content'], tf.uint8)
        content_image.set_shape([48 * 48 * 4])
        content_image = tf.reshape(content_image, [48, 48, 4])[:,:,:CHANNELS]
        # Normalize the values of the image from [0, 255] to [-1.0, 1.0]
        content_image = tf.cast(content_image, dtype=tf.float32) / 127.5 - 1

        style_image = tf.decode_raw(features['image_style'], tf.uint8)
        style_image.set_shape([48 * 48 * 4])
        style_image = tf.reshape(style_image, [48, 48, 4])[:,:,:CHANNELS]
        # Normalize the values of the image from [0, 255] to [-1.0, 1.0]
        style_image = tf.cast(style_image, dtype=tf.float32) / 127.5 - 1
        
        label = features['label']

        return {'style_images': style_image, 'content_images': content_image, 'labels': label}, label


def make_input_fn(is_training=True, one_batch=False):
    def input_fn(params):
        batch_size = params['batch_size']
        dataset = tf.data.TFRecordDataset(data_train_file, buffer_size=8*1024*1024)
        dataset = dataset.map(parser).cache().shuffle(batch_size)
        if is_training:
            dataset = dataset.repeat()
        if one_batch:
            dataset = dataset.take(params['batch_size'])
            
        dataset = dataset.batch(batch_size, drop_remainder=True).prefetch(8)

        return dataset
    return input_fn


def predict_input_fn(params):
    batch_size = params['batch_size']
    dataset = tf.data.TFRecordDataset(data_test_file).map(parser).cache().shuffle(batch_size)
    dataset = dataset.prefetch(batch_size).batch(batch_size, drop_remainder=True)
    
    return dataset

In [0]:
############################### DATA SAVEING ###################################
 
def images_to_zero_one(images):
        return np.clip(np.array(images) * 0.5 + 0.5, 0., 1.)


def save_imgs(epoch, images):
    if not os.path.exists(GOOGLE_DRIVE_DIR):
        os.mkdir(GOOGLE_DRIVE_DIR)

    # Rescale images to 0 - 1
    images = images_to_zero_one(images)
    fig, axs = plt.subplots(R, C, figsize=(20,20))

    for i in range(R):
        for j in range(C):
            axs[i,j].imshow(images[C*i + j])
            axs[i,j].axis('off')

    fig.savefig(os.path.join(GOOGLE_DRIVE_DIR, '{}.png'.format(epoch)))
    plt.close()

In [0]:
################################## MODEL #######################################

def _relu(x):
    return tf.nn.relu(x)


def _leaky_relu(x):
    return tf.nn.leaky_relu(x, alpha=ALPHA)


def _instance_norm(x):
    return tf.contrib.layers.instance_norm(x)


def _dense(x, neurons, name, activation=None):
    return tf.layers.dense(x, neurons, name=name, activation=activation,
                           kernel_initializer=tf.truncated_normal_initializer(stddev=0.02))


def _conv2d(x, filters, kernel_size, stride, name, activation=None):
    return tf.layers.conv2d(x, filters, [kernel_size, kernel_size], 
                            strides=[stride, stride], activation=activation,
                            padding='same', name=name,
                            kernel_initializer=tf.truncated_normal_initializer(stddev=0.02))

def _pooling(x, kernel, stride, index):
    return tf.layers.average_pooling2d(x, pool_size=kernel, strides=stride, name='pool_{}'.format(index))

def _deconv2d(x, filters, stride, name, activation=None):
    return tf.layers.conv2d_transpose(x, filters, [KERNEL_SIZE, KERNEL_SIZE],
                                      strides=[stride, stride], activation=activation,
                                      padding='same', name=name,
                                      kernel_initializer=tf.truncated_normal_initializer(stddev=0.02))


def convolution_block(x, filters, kernel_size, resize_factor, index, activation=_relu, activation_first=False, normalization=None):
    if activation and activation_first:
        x = activation(x)
    x = _conv2d(x, kernel_size=kernel_size, filters=filters, stride=resize_factor, activation=None, name='conv_{}'.format(index))
    if normalization:
        x = normalization(x)
    if activation and not activation_first:
        x = activation(x)
    return x


def deconvolution_block(x, filters, resize_factor, index, activation=_relu, normalization=None):
    x = _deconv2d(x, filters=filters, stride=resize_factor, activation=None, name='deconv_{}'.format(index))
    if normalization:
        x = normalization(x)
    if activation:
        x = activation(x)
    return x


def fully_connected_block(x, neurons, index, activation=_relu, normalization=None):
    x = _dense(x, neurons=neurons, activation=None, name='fc_{}'.format(index))
    if normalization:
        x = normalization(x)
    if activation:
        x = activation(x)
    return x


def residual_block(x, filters_in, filters_out, index, kernel_size=3, activation=_relu, activation_first=False, normalization=None):
    forwarded_x = convolution_block(x, filters_in, kernel_size, resize_factor=1, 
                                    index='res_con1_{}'.format(index), normalization=normalization,
                                    activation_first=activation_first, activation=activation)
    
    forwarded_x = convolution_block(forwarded_x, filters_out, kernel_size, resize_factor=1, 
                                    index='res_con2_{}'.format(index), normalization=normalization,
                                    activation_first=activation_first, activation=None)
    
    if filters_in != filters_out:
        x = convolution_block(x, filters_out, kernel_size=1, resize_factor=1, index='res_con3_{}'.format(index), activation=None)
    x = tf.add(forwarded_x, x)
    return x


class Funit:

    @staticmethod
    def discriminator(x, label, scope='Discriminator'):
        with tf.variable_scope(scope, reuse=tf.AUTO_REUSE):
            x = convolution_block(x, 64, kernel_size=7, resize_factor=1, index='disc_11')
            
            x = residual_block(x, 64, 64, 'disc_12', kernel_size=3, activation=_leaky_relu, activation_first=True)
            x = residual_block(x, 64, 128, 'disc_13', kernel_size=3, activation=_leaky_relu, activation_first=True)
            x = _pooling(x, 3, 2, 'disc_14')
            
            x = residual_block(x, 128, 128, 'disc_15', kernel_size=3, activation=_leaky_relu, activation_first=True)
            x = residual_block(x, 128, 256, 'disc_16', kernel_size=3, activation=_leaky_relu, activation_first=True)
            x = _pooling(x, 3, 2, 'disc_17')
            
            x = residual_block(x, 256, 256, 'disc_18', kernel_size=3, activation=_leaky_relu, activation_first=True)
            x = residual_block(x, 256, 512, 'disc_19', kernel_size=3, activation=_leaky_relu, activation_first=True)
            x = _pooling(x, 3, 2, 'disc_20')
            
            x = residual_block(x, 512, 512, 'disc_21', kernel_size=3, activation=_leaky_relu, activation_first=True)
            x = residual_block(x, 512, 1024, 'disc_22', kernel_size=3, activation=_leaky_relu, activation_first=True)
            x = _pooling(x, 3, 2, 'disc_23')
            
            x = residual_block(x, 1024, 1024, 'disc_24', kernel_size=3, activation=_leaky_relu, activation_first=True)
            x = residual_block(x, 1024, 1024, 'disc_25', kernel_size=3, activation=_leaky_relu, activation_first=True)
            
            output = tf.layers.flatten(x)
            output = _dense(output, NUM_CLASSES, 'disc_out')
#             output = tf.gather_nd(output, tf.expand_dims(label, axis=1), batch_dims=1)
            
            return output, x

In [0]:
################################ MODEL FUN #####################################
def make_model_fn(model):

    def model_fn(features, labels, mode, params):

        with tf.variable_scope('inputs'):
            style_images = features['style_images']
            labels = features['labels']
        d_on_data_labels = tf.one_hot(labels, depth=NUM_CLASSES, axis=-1)
        logits, _ = model.discriminator(style_images, labels)
        
        # PREDICT #
        if mode == tf.estimator.ModeKeys.PREDICT:
          
            predictions = {
                'd_style': tf.argmax(tf.sigmoid(logits), axis=-1),
                'd_style_one_hot': tf.sigmoid(logits),
                'one_hot_labels': d_on_data_labels,
                'labels': labels
            }
            return tf.contrib.tpu.TPUEstimatorSpec(mode=mode, predictions=predictions)

          
        # Discriminator loss
        
        print()
        d_on_data_logits = tf.squeeze(logits)
        d_loss = tf.losses.sigmoid_cross_entropy(
            d_on_data_labels,
            d_on_data_logits,
            reduction=tf.losses.Reduction.NONE,
        )

        # TRAIN #
        if mode == tf.estimator.ModeKeys.TRAIN:
            d_loss = tf.reduce_mean(d_loss)
            d_optimizer = tf.train.AdamOptimizer(learning_rate=D_LR, beta1=0.5)
            d_optimizer = tf.contrib.tpu.CrossShardOptimizer(d_optimizer)
         
            with tf.control_dependencies(tf.get_collection(tf.GraphKeys.UPDATE_OPS)):
                d_step = d_optimizer.minimize(d_loss, var_list=tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES,
                                                                                 scope='Discriminator'))
                increment_step = tf.assign_add(tf.train.get_or_create_global_step(), 1)
                joint_op = tf.group([d_step, increment_step])

                a = tf.contrib.tpu.TPUEstimatorSpec(mode=mode, loss=d_loss, train_op=joint_op)
                return a

        # EVAL #
        elif mode == tf.estimator.ModeKeys.EVAL:
            def _eval_metric_fn(d_loss, d_real_labels, d_real_logits):
                return {
                    'discriminator_loss': tf.metrics.mean(d_loss),
                    'discriminator_real_accuracy': tf.metrics.accuracy(labels=d_real_labels, predictions=tf.math.argmax(tf.sigmoid(d_real_logits), axis=-1)),
                }

            return tf.contrib.tpu.TPUEstimatorSpec(mode=mode, loss=tf.reduce_mean(d_loss),
                                                   eval_metrics=(_eval_metric_fn, [d_loss, labels, d_on_data_logits]))
    return model_fn

In [0]:
################################ ESTIMATORS ####################################

def make_estimators(model, only_cpu=False):
    model_fn = make_model_fn(model)
    
    config = tf.contrib.tpu.RunConfig(
        master=TF_MASTER,
        save_checkpoints_steps=EVAL_EPOCHS,
        save_checkpoints_secs=None,
        save_summary_steps=EVAL_EPOCHS,
        model_dir=MODEL_DIR,
        keep_checkpoint_max=3,
        tpu_config=tf.contrib.tpu.TPUConfig(iterations_per_loop=EVAL_EPOCHS))

    if not only_cpu:
        # TPU-based estimator used for TRAIN and EVAL
        est = tf.contrib.tpu.TPUEstimator(
            model_fn=model_fn,
            use_tpu=True,
            config=config,
            train_batch_size=BATCH_SIZE,
            eval_batch_size=BATCH_SIZE)
    else:
        est = None

    # CPU-based estimator used for PREDICT (generating images)
    cpu_est = tf.contrib.tpu.TPUEstimator(
        model_fn=model_fn,
        use_tpu=False,
        config=config,
        predict_batch_size=EXAMPLES)
    
    return est, cpu_est

In [0]:
################################# TRAINING #####################################

def train(est, cpu_est):
    current_step = estimator._load_global_step_from_checkpoint_dir(MODEL_DIR)
    print('Starting training')

    while current_step < EPOCHS:
        next_checkpoint = int(min(current_step + EVAL_EPOCHS, EPOCHS))
        est.train(input_fn=make_input_fn(), max_steps=next_checkpoint)
        current_step = next_checkpoint
        print('Finished training step %d' % current_step)

        # Evaluation
        metrics = est.evaluate(input_fn=make_input_fn(False), steps=1)
        print('Finished evaluating')
        print(metrics)
        
        generated_iter = cpu_est.predict(input_fn=make_input_fn(False, True))
        for p in generated_iter:
            print (p['labels'], p['one_hot_labels'], p['d_style'], p['d_style_one_hot'])
        

#         Render some generated images
#         generated_iter = cpu_est.predict(input_fn=predict_input_fn)
#         images = [np.concatenate([p['content_images'], p['style_images'], p['generated_images']], axis=1) for p in generated_iter]
#         save_imgs(str(current_step), images)
#         print('Finished generating images')

In [0]:
def do_experiment():
    setup()
    upload_credentials()
    model = Funit()
    est, cpu_est = make_estimators(model)
    train(est, cpu_est)

In [0]:
tf.logging.set_verbosity(tf.logging.INFO)

try:
    do_experiment()
except Exception as e:
    print (e)
    pass

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


W0905 12:01:30.829385 140716071708544 estimator.py:1984] Estimator's model_fn (<function make_model_fn.<locals>.model_fn at 0x7ffaca2d7840>) includes params argument, but params are not passed to Estimator.
I0905 12:01:30.831735 140716071708544 estimator.py:209] Using config: {'_model_dir': 'gs://tputestingmnist/FUNIT/FUNIT_only_disc_1/', '_tf_random_seed': None, '_save_summary_steps': 1000, '_save_checkpoints_steps': 1000, '_save_checkpoints_secs': None, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 3, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': None, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_experimental_max_worker_delay_secs': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7ffab99e0cc0>, '_task_type': 'worker', '_task_i

Starting training



I0905 12:01:35.499120 140716071708544 basic_session_run_hooks.py:541] Create CheckpointSaverHook.
I0905 12:01:35.606193 140716071708544 estimator.py:1147] Done calling model_fn.
I0905 12:01:37.306087 140716071708544 tpu_estimator.py:499] TPU job name tpu_worker
I0905 12:01:37.859925 140716071708544 monitored_session.py:240] Graph was finalized.
I0905 12:01:39.530585 140716071708544 session_manager.py:500] Running local_init_op.
I0905 12:01:39.657754 140716071708544 session_manager.py:502] Done running local_init_op.
I0905 12:01:41.588216 140716071708544 basic_session_run_hooks.py:606] Saving checkpoints for 0 into gs://tputestingmnist/FUNIT/FUNIT_only_disc_1/model.ckpt.
I0905 12:01:53.171544 140716071708544 util.py:98] Initialized dataset iterators in 0 seconds
I0905 12:01:53.173085 140716071708544 session_support.py:332] Installing graceful shutdown hook.
I0905 12:01:53.178188 140716071708544 session_support.py:82] Creating heartbeat manager for ['/job:tpu_worker/replica:0/task:0/devi

Finished training step 1000



...

Finished evaluating
{'discriminator_loss': 0.1898317, 'discriminator_real_accuracy': 0.078125, 'loss': 0.18999855, 'global_step': 12000}


I0905 12:37:23.210506 140716071708544 estimator.py:1145] Calling model_fn.
I0905 12:37:23.211974 140716071708544 tpu_estimator.py:2965] Running infer on CPU
I0905 12:37:23.662814 140716071708544 estimator.py:1147] Done calling model_fn.
I0905 12:37:23.836031 140716071708544 monitored_session.py:240] Graph was finalized.
I0905 12:37:23.885420 140716071708544 saver.py:1280] Restoring parameters from gs://tputestingmnist/FUNIT/FUNIT_only_disc_1/model.ckpt-12000
I0905 12:37:24.250349 140716071708544 session_manager.py:500] Running local_init_op.
I0905 12:37:24.272865 140716071708544 session_manager.py:502] Done running local_init_op.
I0905 12:37:24.600731 140716071708544 error_handling.py:96] prediction_loop marked as finished
I0905 12:37:24.602025 140716071708544 error_handling.py:96] prediction_loop marked as finished


7 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 17 [0.04341033 0.04955021 0.04363334 0.03912821 0.05654418 0.05010247
 0.04498032 0.04468298 0.04513991 0.04630175 0.06039983 0.03577879
 0.04001892 0.04199809 0.04322773 0.05880547 0.04122674 0.06577477
 0.04143345 0.04321587 0.04853609]
16 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0.] 1 [0.03608698 0.05063754 0.03965807 0.03596127 0.04502928 0.02799651
 0.04231036 0.03857672 0.03838891 0.03898817 0.0495227  0.03642052
 0.04392797 0.0421454  0.0391345  0.04469323 0.03731519 0.05030763
 0.03982425 0.03687599 0.04892725]
11 [0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 11 [0.03672099 0.06199834 0.04280859 0.04419655 0.04569209 0.06476772
 0.04256585 0.03829107 0.03458115 0.04110041 0.04174986 0.07398787
 0.05201131 0.04168063 0.03070799 0.04003289 0.03687042 0.0464339
 0.04208103 0.0372141  0.06088626]
6 [0. 0. 0. 0. 0. 0. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.] 5 [0.03079483 0

I0905 12:37:24.960414 140716071708544 estimator.py:1145] Calling model_fn.


I0905 12:37:28.373460 140716071708544 basic_session_run_hooks.py:541] Create CheckpointSaverHook.
I0905 12:37:28.470349 140716071708544 estimator.py:1147] Done calling model_fn.
I0905 12:37:28.471949 140716071708544 tpu_estimator.py:499] TPU job name tpu_worker
I0905 12:37:29.027841 140716071708544 monitored_session.py:240] Graph was finalized.
I0905 12:37:29.141673 140716071708544 saver.py:1280] Restoring parameters from gs://tputestingmnist/FUNIT/FUNIT_only_disc_1/model.ckpt-12000
I0905 12:37:30.523337 140716071708544 session_manager.py:500] Running local_init_op.
I0905 12:37:30.651082 140716071708544 session_manager.py:502] Done running local_init_op.
